# Activities

Note: the practice happens almost exclusively on paper, potentially augmented by multiple choice questions in order to submit an answer that shows understanding.

The activities performed are:
- 1 - Build a single RESTful controller against an EF database (A)
- 2 - Add pagination to `GetAll` (M)
- 3 - Add generic predicate filtering to `GetAll` (M)

### 1 Building a RESTful controller and test it with POSTMAN
- To start the implementation you need two addtional tools: 
    - A HTTP client named `Postman` that send and recieve HTTP messages (json) to and from a web API.  Download Postman:(https://www.getpostman.com/).

    - A .net core package called `Newtonsoft.Json` to serialize json messages and deserialize .NET object. This package will be added after project creation.


- In order to autmatically generate a controller in dotnet core we will create a `webapi-application`.  Then we add the `model` part to it (see step 3). To setup a webapi-project in a folder called lesson3 excute the following command: 

`dotnet new webapi -o lesson3`

Go to the folder of your project type those commands in the command line:

`dotnet add package Newtonsoft.Json`
`dotnet restore`

You can compile and run this project to check if the web API is working by default. After excuting the run-command `dotnet run` in the console your web API will start working by showing the message: 

`Now listening on: http://localhost:5000`
`Application started. Press Ctrl+C to shut down.`



####  Step 2: Web API check

- By default the .net core will generate a controller for you in the root folder. Go to the controller-folder and open the file `ValuesController.cs`. 

```
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers
{
    //This is the default route of the API. 
    [Route("api/[controller]")]
    public class ValuesController : Controller
    {
        // GET api/values
        [HttpGet]
        public IEnumerable<string> Get()
        {
            return new string[] { "value1", "value2" };
        }

        // GET api/values/5
        [HttpGet("{id}")]
        public string Get(int id)
        {
            return "value";
        }

        // POST api/values
        [HttpPost]
        public void Post([FromBody]string value)
        {
        }

        // PUT api/values/5
        [HttpPut("{id}")]
        public void Put(int id, [FromBody]string value)
        {
        }

        // DELETE api/values/5
        [HttpDelete("{id}")]
        public void Delete(int id)
        {
        }
    }
}
```

- By default the web API will have the uri: `http://localhost:5000/api/values`. To check the communication between this API and a client we will use `Postman`. After starting `Postman` Select the command `GET`; enter the uri mentioned above; and click on `send-button`. After few seconds you should recieve a HTTP message from your api in json-format. the default message should be: 

`{"value1", "value2"}`.
 
#### step 3: Add a model

Now we need to adapt the default controller (ValuesController.cs) to get it working with our data storage and the model from practicum1. 

- You need to make few adjustments to get that model working in your current project setup. 
    - Create a folder named `Models`
    - Copy the `Models.cs` file from your previous project (practicum1) to the `Models` folder of your current project. 
    - Change the namespace to match the following:

`namespace lesson3.Models`

- The files that needs to be adjusted from now on are: 
    - `Models.cs` in the Model folder
    - `Startup.cs` in the root directory of your project
    - `ValuesController` in the Controller folder 
 
- Change the class `MovieContext` in `Models.cs` to match this structure: 

```
...
namespace lesson3.Models
{
 public class MovieContext : DbContext
    {

        public DbSet<Actors> Actors { get; set; }
        public DbSet<Movies> Movies { get; set; }
        
        //Added constructor to provide the connection to the database as a service (look at: startup.cs)
        public MovieContext(DbContextOptions<MovieContext> options): base(options)
        {
        }
        
        //Uncomment or remove the OnConfiguring method. It is not need any more
        // protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
        // {
        //     if (!optionsBuilder.IsConfigured)
        //     {
        //
        //         // http://go.microsoft.com/fwlink/?LinkId=723263 for guidance on storing connection strings
        //         optionsBuilder.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres");
        //     }
        // }
...
}
```

- The connection to the database will be provided as a service in this project. You need to add the connection string  to the `ConfigureServices` method in `Startup.cs`. Remember to change the connection parameters.

```
...

// This method gets called by the runtime. Use this method to add services to the container.
public void ConfigureServices (IServiceCollection services) {

        //Add this line to your method
     services.AddDbContext<MovieContext> (
                 opt => opt.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres"));
     
     services.AddMvc ();

        }
...

```

- Change the name `ValuesController.cs` to `MoviesController.cs`.  We need to add a private property for the context to access it from our controller. This property will be intilized in the contructor. To understand how the context can be used in the controller the method `Get()` and `Get(int id)` are implemented to get the movies as a result form the database. 

```

using System;
using System.Collections.Generic;
using System.Linq;
using System.Linq.Expressions;
using System.Threading.Tasks;
using lesson3.Models;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers {

    [Route ("api/[controller]")]
    public class MoviesController : Controller {
        private readonly MovieDBContext _context;

        public MoviesController (MovieDBContext context) {
            _context = context;
        }

        // GET api/values
        [HttpGet]
        public IEnumerable<Movies> Get () {
            return _context.Movies.ToList ();;
        }

        // GET api/values/5
        [HttpGet ("{id}")]
        public IActionResult Get (int id) {
            var movie = _context.Movies.FirstOrDefault (t => t.MovieId == id);
            if (movie == null) {
                return NotFound ();
            }
            return new ObjectResult (movie);
        }

        // POST api/values
        [HttpPost]
        public void Post ([FromBody] Movies value) { }

        // PUT api/values/5
        [HttpPut ("{id}")]
        public void Put (int id, [FromBody] Movies value) { }

        // DELETE api/values/5
        [HttpDelete ("{id}")]
        public void Delete (int id) { }
    }
}

```


### 2 Adding paginations

### 3 Generic predicate filtering
To use generic predicates to select entities of our database we need use LINQ dynamically by means of `Expression Trees` [1]. Expression trees allow to build/compose dynamically LINQ objects that can be used to test for example whether to select or not an entity, to order the result of a query, etc.



In the following a schematized version of a simple application is presented. In the schema a client is trying to get all the actors from the remote database that are called `Pippo` and which ages are 18. To do so the client first builds a filter expression (**1**) that captures the just mentioned conditions, and then calls (**2**) the remote method `GetAll` to which the just built filter expression (_our predicate_) is passed as a json string. 



The controller captures the call request (**3**) and forwards it to the right method (**4**). The method `GetAll` converts the input filter expression to an equivalent version in `Expression Tree`. The converted expression and the rest of the query is then passed to EF (**5**) which takes will take care of selecting the right actors. Once selected, the actors are returned back to the client (**6**). The client, which meanwhile was waiting for a response (**2.1**) receives the actors and saves them for future usages (**7**).



<img src="_1.jpg">



In the following the code for the client and the server is provided and commented. Use the above image to follow the structure of the solution.

#### Client
For this solution we will use a client written in C# (you can skip this part and use directly postman; in this case make sure to build correctly the body of the predicate to send via http). 


Create a a console application `dotnet new console` and add the Newtonsoft package to it `dotnet add package Newtonsoft.Json`. Newtonsoft is needed to parse the predicate into a json structure to send to the server. In the end the code should look like the following:


<img src="_2.jpg">

The classes needed for the correct functioning of the code above are:

<img src="_3.jpg">


Before you run the program make sure that the server (which definition is provided in the following section) is running.

#### Server

To implement the server we need to make a _webapi_ project (`dotnet new webapi`). Once created, add to it the design package (`dotnet add package Microsoft.EntityFrameworkCore.Design`), and by hand add a reference to the EntityFrameworkCore in the projects' setting file (the one that ends in *.csproj).


In the end the project setting file should look like this:

<img src="_4.jpg">

Once created create a folder Models and add the models from the previous practicums. You need to change for this implementaiton the constructor of the context. It should eventually look like the following (in this example we use the 1.Movie-N.Actors example):


<img src="_5.jpg">

Modify the Startup.cs file to connect our model to the controller once we run our server.

<img src="_6.jpg">


In the _Controllers_ folder add a MovieController.cs file. In this file we will configure the controller, which will take care of providing our movies and actors to the client.

In the just made MovieController file add the folowing usings:

<img src="_7.jpg">

Copy the _Filter_ class from the clients' code. We need this class, because the deserializer in our server needs to know how to build-back the seriealized predicate. 


<img src="_8.jpg">

Add a MovieController class which contrcutor will take care of saving locally the database context, and filling the database with some initial values (for debugging).

<img src="_9.jpg">

Add now the controller that will return the actors that satisfy a certain predicate

<img src="_10.jpg">

To conclude in the following the method that converts our filter predicate to an expression tree is shown.


<img src="_11.jpg">

To run the server use the command `dotnet run`. Now test it by calling run on the client written above.


References:
- [1] https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/expression-trees/
